# EuroPython program grid

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 99999;
//increase max size of output area

<IPython.core.display.Javascript object>

In [1]:
import json
import datetime as dt
from collections import OrderedDict, defaultdict
from random import choice, randrange, shuffle
from copy import deepcopy

from itertools import product
from functools import partial
from operator import itemgetter

from eptools.dict_query import build_query, run_query

from IPython.display import display, HTML
show = lambda s: display(HTML(s))

In [2]:
talk_sessions = json.load(open('accepted_talks.json'))

In [3]:
list(talk_sessions.keys())

['Lightning talk',
 'training',
 'poster',
 'EPS session',
 'Opening session',
 'Closing session',
 'talk']

In [4]:
#all talks
all_talks = []
for s in talk_sessions.values():
    all_talks.extend(list(s.values()))

#the talks worth for scheduling
grid_talks = []
sessions = talk_sessions.copy()
general_grid_sessions = ['talk', 'training']
for session_name in general_grid_sessions:
    grid_talks.extend(sessions[session_name].values())

fields2pop = ['abstract_extra', 
              'abstract_long',
              'abstract_short',
              'twitters',
              'emails',
              'status',
              'url',
              'companies',
              'have_tickets',
             ]
for talk in grid_talks:
    for f in fields2pop:
        talk.pop(f)

In [5]:
tags_field = 'tag_categories'

weekday_names = {0: 'Monday, July 18th', 
                 1: 'Tuesday, July 19th',
                 2: 'Wednesday, July 20th',
                 3: 'Thursday, July 21st',
                 4: 'Friday, July 22nd'
                }

room_names = {0: 'A1',
              1: 'A3',
              2: 'A2',
              3: 'Ba1',
              4: 'Ba2',
              5: 'E' ,
              6: 'A4',
             } 

# this is not being used yet
durations = {'announcements': 15,
             'keynote': 45,
             'lts': 60,
             'lunch': 60,
             'am_coffee': 30,
             'pm_coffee': 30,
            }

# track schedule types, by talk conditions
track_schedule1 = [(('duration', 45), ),
                   (('duration', 45), ),
                   (('duration', (45, 60)), ),
                   (('duration', 45), ),
                   (('duration', 45), ),
                   (('duration', 45), ),
                   (('duration', 45), ),
                   (('admin_type', 'Lightning talk')),
                  ]

track_schedule2 = [(('duration', 45), ),
                   (('duration', 45), ),
                   (('duration', (45, 30)), ),
                   (('duration', 45), ),
                   (('duration', 45), ),
                   (('duration', 30), ),
                   (('duration', 30), ),
                  ]

track_schedule3 = [(('duration', 45), ), 
                   (('duration', 45), ),
                   (('duration', 30), ),
                   (('duration', 60), ),
                   (('duration', 45), ),
                   (('duration', (60, 30)), ),
                   (('duration', 30), ),
                  ]

#tutorials
track_schedule4 = [('type', 'Training (180 mins)'), 
                   ('type', 'Training (180 mins)'), ]


# these are for reference, but not being taken into account (yet)
frstday_schedule1 = [(('admin_type', 'Opening session')), 
                     (('admin_type', 'Keynote')),
                    ] + track_schedule1

lastday_schedule1 = track_schedule1 + [(('admin_type', 'Closing session')),]

# I removed time from here.
#daily_timegrid = lambda schedule: OrderedDict([(datetime.time(*slot[0]), slot[1]) for slot in schedule])

room1_schedule = track_schedule1 # A1, the google room
room2_schedule = track_schedule2 # A3, pythonanywhere room
room3_schedule = track_schedule3 # A2
room4_schedule = track_schedule3 # Barria1
room5_schedule = track_schedule3 # Barria2

room6_schedule = track_schedule4 # Room E
room7_schedule = track_schedule4 # Room A4

daily_schedule   = OrderedDict([(0, room1_schedule), 
                                (1, room2_schedule),
                                (2, room3_schedule),
                                (3, room4_schedule),
                                (4, room5_schedule),
                                (5, room6_schedule),
                                (6, room7_schedule)])

# week conditions
default_condition = [('language', 'English')]

# [day][room] -> talk conditions
dayroom_conditions = {0: {6: [('language', 'Spanish')], },
                      1: {4: [('language', 'Spanish')], },
                      2: {3: [('language', 'Basque')],  },
                      3: {},    
                      4: {},    
                     }

In [6]:
# the whole schedule conditions table
week_conditions = defaultdict(dict)
for day, room in product(weekday_names, room_names):
    week_conditions[day][room] = daily_schedule.get(day, []) + dayroom_conditions[day].get(room, [])

In [7]:
def find_talk(talk_title):
    return [talk for talk in all_talks if talk_title in talk['title']]

find_talk("So, what's all the fuss about Docker?")

[{'adm_type': '',
  'duration': 180,
  'id': 643,
  'language': 'English',
  'level': 'Intermediate',
  'speakers': 'Daniele Procida',
  'sub_community': '',
  'sub_title': 'A hands-on workshop to familiarise Django developers with key Docker concepts and techniques',
  'tag_categories': ['Application Frameworks', 'DevOps'],
  'tags': ['Django', 'Docker'],
  'timerange': '2016-07-22 14:00:00, 2016-07-22 17:00:00',
  'title': "So, what's all the fuss about Docker?",
  'track_title': 'A4',
  'type': 'Training (180 mins)'}]

In [8]:
tags2pop = ['>>> suggested track', 'python', '']
tags = {}
text = []
for talk in all_talks:
    for t in talk[tags_field]:
        if t.lower() in tags2pop:
            continue            
        if not t.lower() in tags:
            tags[t.lower()] = 0
        tags[t.lower()] += 1
        text.append(t.lower())
tags_sorted = sorted(tags.items(), key=itemgetter(1), reverse=True)
tags_sorted

[('programming', 94),
 ('data science', 88),
 ('devops', 64),
 ('best practice and use cases', 48),
 ('educational', 29),
 ('web', 27),
 ('community', 25),
 ('testing', 24),
 ('open source', 17),
 ('databases', 15),
 ('everything else', 14),
 ('hardware', 13),
 ('security', 12),
 ('other programming languages', 11),
 ('case study', 11),
 ('sciences', 10),
 ('application frameworks', 8),
 ('operating systems', 2),
 ('development methods', 2),
 ('business', 1)]

In [9]:
def pick_talk(talks, tags, query, duration=45):
    if not talks:
        raise IndexError('The list of talks is empty!')

    while tags:
        randidx = randrange(0, len(tags))
        atag    = tags.pop(randidx)[0]
        for tlkidx, talk in enumerate(talks):
            talk_tags = [t.lower() for t in talk[tags_field]]
            if query(talk) and atag in talk_tags:
                atalk = talks.pop(tlkidx)
                return atalk
    
    # if no talk fills the query requirements
    return pick_talk_by_duration(talks, duration)
            
            
def pick_talk_by_duration(talks, duration):
    if not talks:
        raise IndexError('The list of talks is empty!')

    for tlkidx, talk in enumerate(talks):
        if talk['duration'] == duration:
                atalk = talks.pop(tlkidx)
                return atalk
    return {}

In [10]:
import random

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]
        

def split(xs, n):
    ys = list(xs)
    random.shuffle(ys)
    size = len(ys) // n
    leftovers= ys[size*n:]
    for c in range(n):
        if leftovers:
           extra = [ leftovers.pop() ] 
        else:
           extra = []
        yield ys[c*size:(c+1)*size] + extra

In [11]:
talks = grid_talks.copy()
shuffle(talks)

# random pick talks
week_slots = defaultdict(dict)
for day in weekday_names:
    tag_chunks = split([t[0] for t in tags_sorted], len(room_names))
    room_tags  = {room: tags for room, tags in zip(room_names.keys(), tag_chunks)}  

    for room in room_names:
         conds = week_conditions[day][room]
#         for slot_cond 
#         query = build_query(conds)
#         try:
#             week_slots[day][room] = pick_talk(talks, tags_sorted, query)
#         except IndexError:
#             print('No talks left for {}.'.format(query))
#         except:
#             raise

In [176]:
weekly_schedule = OrderedDict()

for d in range(n_days):
    weekly_schedule[d] = deepcopy(daily_schedule)
    day_talks          = week_talks[d].copy()
    for room in daily_schedule:
        for slot in daily_schedule[room]:
            #print(weekly_schedule[d][room].keys())
            # TODO: build condition
            # TODO: query for talk
            weekly_schedule[d][room][slot] = day_talks.pop(0)
            #print(day_talks)

TypeError: list indices must be integers or slices, not tuple

In [248]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)


def tabulate(time_list, header=''):
    table = ListTable()
    table.append(header)
    for slot in time_list:
        table.append([slot] + time_list[slot])
    return table


def get_room_schedule(weekly_schedule, room_name, field='title'):
    slots = list(daily_schedule[room_name].keys())
    daily_slots = []
    for slot in slots:
        talks = [weekly_schedule[d][room_name][slot].get(field, '-') for d in range(n_days)]
        daily_slots.append((slot, talks))
    room_schedule = OrderedDict(daily_slots)
    return room_schedule


from itertools import zip_longest
def get_day_schedule(weekly_schedule, day_num, field='title'):
    day_schedule = weekly_schedule[day_num]
    nslots = max([len(slots) for room, slots in ds.items()])
    room_slots = []
    for room, talk_slots in day_schedule.items():
        room_talks = [talk.get(field, '-') for slot, talk in talk_slots.items()]
        room_slots.append(room_talks)
    schedule = OrderedDict(list(enumerate(list(map(list, zip_longest(*room_slots))))))
    return schedule

In [249]:
show('<h1>Not scheduled talks</h1>')

for talk in talks:
    print(talk)

# TALKS

In [259]:
for day, _ in enumerate(weekday_names):
    show('<h1>{}</h1>'.format(weekday_names[day]))
    show(tabulate(get_day_schedule(weekly_schedule, day), header=['Slot'] + list(room_names.values()))._repr_html_())

Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,Introduction to aiohttp,Un vector por tu palabra,CloudABI: Capability based security on Linux/UNIX,"OMG, Bokeh is better than ever!",FAT Python: a new static optimizer for Python 3.6,uWSGI: the full stack application server,The Stupid Python Workshop
1,Against the silos: usable encrypted email & the quest for privacy-aware services,High Performance Networking in Python,Things I wish I knew before starting using Python for Data Processing,"Beyond scraping, getting data from dynamic, heavily javascript driven, websites",Building your own AI,Python for System Administrators,Introduction to Deep Learning for Natural Language Processing
2,What is the best full text search engine for Python?,Get Instrumented!,"Endor, ipuinak kontatzen zituen Nao robota.","Python, Data & Rock'n'Roll",Conda - Easier Installs and Simpler Builds,None,None
3,Hacking ético con herramientas Python,Jupyter for everything else,APIs and Microservices With Go,EPS General Assembly,A deep dive into the Pymongo MongoDB driver,None,None
4,Using and abusing Python’s double-underscore methods and attributes,An Introduction to Deep Learning,Python as the keystone of building and testing C++ applications,System Testing with pytest and docker-py,MicroPython on the BBC micro:bit,None,None
5,Real virtual environments without virtualenv,Test Driven Deployment with Ansible 2.0,Behind Closed Doors: Managing Passwords in a Dangerous World,Create secure production environment using Docker,Clean code in Python,None,None
6,Go for Python Programmers,None,The Journey from Python Developer to Python Company Owner,Implementación de un Identificador de Sonido en Python,Monkey-patching: a magic trick or a powerful tool?,None,None


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,Moving away from NodeJS to a pure python solution for assets,Kung Fu at Dawn with Itertools,Do I need to switch to Go(lang) ?,The Joy of Simulation: for Fun and Profit,Raspberry Pi GPIO Zero,Present-day Async Web development training: from Twisted to Tornado and AsyncIO,Efficient Python for High-Performance Parallel Computing
1,Using Service Discovery to build dynamic python applications,Effective Code Review,TDD of Python microservices,How OpenStack makes Python better (and vice-versa),Data Formats for Data Science,Guide to make a real contribution to an open source project for novice,"So, what's all the fuss about Docker?"
2,Profiling the unprofilable,"Infrastructure as Code: ""pip install"" your environment",EITB Nahieran: askatu bideoak API honen bidez,Simplifying Computer Art in Python,Python and Async programming,None,None
3,Scaling Microservices with Crossbar.io,How to improve your diet and save money with Python,Modern OpenGL with Python,Nipy on functional brain MRI,-,None,None
4,Writing Redis in Python with asyncio,PSF Meeting,Hands-on with nilearn for Neuroimaging,Building beautiful RESTful APIs using Flask,Peeking into Python’s C API,None,None
5,"Server for IoT devices and Mobile devices using Wifi Network,",Ingesting 35 million hotel images with python in the cloud.,"Iteration, iteration, iteration",Datu bistaratze soluzioen garapena Smartcity proiektuetan,How to use Metaclasses to improve your Software Design,None,None
6,Machine Learning: Power of Ensembles,None,Writing faster Python,Entendiendo Unicode,Effectively test your webapp with Python and Selenium,None,None


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,What Python can learn from Haskell packaging,Ethical hacking with Python tools,Analyzing Data with Python & Docker,"I Hate You, NLP... ;)",Building Service interfaces with OpenAPI / Swagger,OpenStack Cloud Native Deployment for Application Developers,"(Machine-)Learning Chinese, with Python!"
1,Python in Astronomy,Operating on Encrypted Data with ZeroDB,Towards More Secure Emailing,"Automate, contribute, repeat.",Deep Learning with Python & TensorFlow,NumPy with Cython,Introduction to Data Wrangling
2,It's not magic: descriptors exposed,What's the point of Object Orientation?,Machine Learning for dummies with Python,So you think your Python startup is worth $10 million...,Pure Data and a Clean Architecture,None,None
3,SQLAlchemy as the backbone of a Data Science company,Metaclasses for fun and profit: Making a declarative GUI implementation,-,-,-,None,None
4,Learn Python The Fun Way,-,Music transcription with Python,import community,Developing a real-time automated trading platform with Python,None,None
5,AFP: secure cloud authentication for machines and humans.,async/await in Python 3.5 and why it is awesome,Is that spam in my ham?,Buildout Django eta Fabric. Kasu praktikoa euskarazko tokiko hedabideetan,Pytest desde las trincheras,None,None
6,NetworkX Visualization Powered by Bokeh,None,Managing Mocks,Dynamic Class Generation in Python,Build your Microservices with ZeroMQ,None,None


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,Efficient Django,Handling GPS Data with Python,Designing a Pythonic Interface,Kung Fu al amanecer con itertools,Build your first OpenStack application with OpenStack PythonSDK,"Faster Python Programs - Measure, don't Guess","Blender: much visual, very 3d, many python."
1,Asynchronous network requests in a web application,Python in Gravitational Waves Research Communities,The Report Of Twisted’s Death,Fast Async Code with Cython and AsyncIO,-,Test-driven code search and the art of writing implementation-agnostic tests,"pytest - simple, rapid and fun testing with Python"
2,Brainwaves for hackers 3,Performant Python,Writing Python Native Extensions in Rust,Building a reasonably popular web application for the first time.,Game Theory to the Rescue When Hard Decisions Are to Be Made,None,None
3,A Gentle Introduction to Neural Networks (with Python),Implementing Parallel Programming Design Patterns using EFL for Python,-,-,-,None,None
4,Get in control of your workflows with Airflow,-,AWS lambda & Python,CFFI: calling C from Python,re-Discovering Python's Regular Expressions,None,None
5,Building a mBaaS framework using Django,Pygame Zero,Predicting Oscar Winners & Box Office Hits with Scikit Learn,MiniBrew: Brewing beer with Python,Another pair of eyes: Reviewing code well,None,None
6,Making robots walk with Python,None,The right way to write documentation,Keeping the Lights on with Python,EuroPython 2017: Help us build the next edition!,None,None


Slot,A1,A3,A2,Ba1,Ba2,E,A4
0,-,-,-,-,-,Productive Coding with PyCharm,Property-based testing with Hypothesis
1,-,-,-,-,-,Manage your Python packages professionally with devpi,IPython in Depth
2,-,-,Wrestling Python into LLVM Intermediate Representation,Python Descriptors for Better Data Structures,Sponsored Talk,None,None
3,-,-,-,-,-,None,None
4,Protect your users with Circuit Breakers,-,Optimize Thyself,Writing unit tests for C code in Python,Sponsored Talk,None,None
5,Implementing a Sound Identifier in Python,RESTful API - Best Practises.,Exploring our Python Interpreter,Exploring Python Bytecode,-,None,None
6,Testing the untestable: a beginner’s guide to mock objects,None,"Grocker, a Python build chain for Docker",The value of mindfulness and how it has arrived at Google,-,None,None


# TRAININGS

### Room E

In [228]:
tabulate(get_room_schedule(weekly_schedule, 'E'), header=[room_names[6]] + weekday_names)

E Room,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,uWSGI: the full stack application server,Present-day Async Web development training: from Twisted to Tornado and AsyncIO,OpenStack Cloud Native Deployment for Application Developers,"Faster Python Programs - Measure, don't Guess",Productive Coding with PyCharm
13:30:00,Python for System Administrators,Guide to make a real contribution to an open source project for novice,NumPy with Cython,Test-driven code search and the art of writing implementation-agnostic tests,Manage your Python packages professionally with devpi


### Room A4

In [164]:
tabulate(get_room_schedule(weekly_schedule, room_names[7]]), header=[room_names[7]]] + weekday_names)

A4 Room,"Monday, July 18th","Tuesday, July 19th","Wednesday, July 20th","Thursday, July 21st","Friday, July 22nd"
11:00:00,The Stupid Python Workshop,Efficient Python for High-Performance Parallel Computing,"(Machine-)Learning Chinese, with Python!","Blender: much visual, very 3d, many python.",Property-based testing with Hypothesis
13:30:00,Introduction to Deep Learning for Natural Language Processing,"So, what's all the fuss about Docker?",Introduction to Data Wrangling,"pytest - simple, rapid and fun testing with Python",IPython in Depth


In [190]:
get_room_schedule(weekly_schedule, 'A1')

OrderedDict([(datetime.time(11, 0),
              ['Introduction to aiohttp',
               'Moving away from NodeJS to a pure python solution for assets',
               'What Python can learn from Haskell packaging',
               'Efficient Django',
               '-']),
             (datetime.time(11, 45),
              ['Against the silos: usable encrypted email & the quest for privacy-aware services',
               'Using Service Discovery to build dynamic python applications',
               'Python in Astronomy',
               'Asynchronous network requests in a web application',
               '-']),
             (datetime.time(12, 30),
              ['What is the best full text search engine for Python?',
               'Profiling the unprofilable',
               "It's not magic: descriptors exposed",
               'Brainwaves for hackers 3',
               '-']),
             (datetime.time(14, 30),
              ['Hacking ético con herramientas Python',
              

## Snippets

In [ ]:
## schedules by room
# tabulate(get_room_schedule(weekly_schedule, 'A1'),  header=['A1'] + weekday_names)
# tabulate(get_room_schedule(weekly_schedule, 'A2'),  header=['A2'] + weekday_names)
# tabulate(get_room_schedule(weekly_schedule, 'A3'),  header=['A3'] + weekday_names)
# tabulate(get_room_schedule(weekly_schedule, 'Ba1'), header=['Barria 1'] + weekday_names)
# tabulate(get_room_schedule(weekly_schedule, 'Ba2'), header=['Barria 2'] + weekday_names)